![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# ONCOLOGY MODELS

This notebook includes details about different kinds of pretrained models to extract oncology-related information from clinical texts, together with examples of each type of model.

## Setup

In [0]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
#nlp.install()

🚨 Your Spark-Healthcare is outdated, installed==4.3.0 but latest version==4.2.4
You can run  jsl.install() to update Spark-Healthcare
🚨 Your Spark-OCR is outdated, installed==4.3.1 but latest version==4.2.1
You can run  jsl.install() to update Spark-OCR


In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL

import os
import json
import string
import numpy as np
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

## List of Pretrained Models

In [0]:
df = pd.DataFrame()
for model_type in ['MedicalNerModel', 'BertForTokenClassification', 'RelationExtractionModel', 'RelationExtractionDLModel', 'AssertionDLModel']:
    model_list = list(set([model[0] for model in medical.InternalResourceDownloader.returnPrivateModels(model_type) if 'oncology' in model[0]]))
    if len(model_list) > 0:
      df = pd.concat([df, pd.DataFrame(model_list, columns = [model_type])], axis = 1)
    
df.fillna('')

MedicalNerModel 
 RelationExtractionModel 
 RelationExtractionDLModel 
 AssertionDLModel 
 
 
 
 
 0 
 ner_oncology_biomarker 
 re_oncology_wip 
 redl_oncology_location_biobert_wip 
 assertion_oncology_family_history_wip 
 
 
 1 
 ner_oncology_biomarker_wip 
 re_oncology_test_result_wip 
 redl_oncology_temporal_biobert_wip 
 assertion_oncology_response_to_treatment_wip 
 
 
 2 
 ner_oncology_therapy_wip 
 re_oncology_biomarker_result_wip 
 redl_oncology_biomarker_result_biobert_wip 
 assertion_oncology_problem_wip 
 
 
 3 
 ner_oncology_unspecific_posology 
 re_oncology_location_wip 
 redl_oncology_granular_biobert_wip 
 assertion_oncology_smoking_status_wip 
 
 
 4 
 ner_oncology_unspecific_posology_healthcare 
 re_oncology_temporal_wip 
 redl_oncology_size_biobert_wip 
 assertion_oncology_demographic_binary_wip 
 
 
 5 
 ner_oncology_anatomy_general_healthcare 
 re_oncology_granular_wip 
 redl_oncology_test_result_biobert_wip 
 assertion_oncology_test_binary_wip 
 
 
 6 
 ner_oncology_anatomy_general 
 re_oncology_size_wip 
 redl_oncology_biobert_wip 
 assertion_oncology_treatment_binary_wip 
 
 
 7 
 ner_oncology_response_to_treatment 
 
 
 assertion_oncology_wip 
 
 
 8 
 ner_oncology_tnm 
 
 
 
 
 
 9 
 ner_oncology_unspecific_posology_wip 
 
 
 
 
 
 10 
 ner_oncology_wip 
 
 
 
 
 
 11 
 ner_oncology_test_wip 
 
 
 
 
 
 12 
 ner_oncology_demographics_wip 
 
 
 
 
 
 13 
 ner_oncology_biomarker_healthcare 
 
 
 
 
 
 14 
 ner_oncology_tnm_wip 
 
 
 
 
 
 15 
 ner_oncology_diagnosis_wip 
 
 
 
 
 
 16 
 ner_oncology_response_to_treatment_wip 
 
 
 
 
 
 17 
 ner_oncology_anatomy_general_wip 
 
 
 
 
 
 18 
 ner_oncology_therapy 
 
 
 
 
 
 19 
 ner_oncology_posology_wip 
 
 
 
 
 
 20 
 ner_oncology_test 
 
 
 
 
 
 21 
 ner_oncology_posology 
 
 
 
 
 
 22 
 ner_oncology_diagnosis 
 
 
 
 
 
 23 
 ner_oncology_demographics 
 
 
 
 
 
 24 
 ner_oncology 
 
 
 
 
 
 25 
 ner_oncology_anatomy_granular 
 
 
 
 
 
 26 
 ner_oncology_anatomy_granular_wip

## NER Models

The NER models from the list include different entity groups and levels of granularity. If you want to extract as much information as possible from oncology texts, then ner_oncology_wip is the best option for you, as it is the most general and granular model. But you may want to use other models depending on your needs (for instance, if you need to extract information related with staging, ner_oncology_tnm_wip would be the most suitable model).

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentence_detector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
 
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")\
    .setSplitChars(["-", "\/"])

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# ner_oncology_wip

ner_oncology_wip = medical.NerModel.pretrained("ner_oncology_wip","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_wip")\

ner_oncology_wip_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_oncology_wip"])\
    .setOutputCol("ner_oncology_wip_chunk")

# ner_oncology_tnm_wip

ner_oncology_tnm_wip = medical.NerModel.pretrained("ner_oncology_tnm_wip","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_tnm_wip")\

ner_oncology_tnm_wip_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_oncology_tnm_wip"])\
    .setOutputCol("ner_oncology_tnm_wip_chunk")

# # ner_oncology_biomarker_wip

ner_oncology_biomarker_wip = medical.NerModel.pretrained("ner_oncology_biomarker_wip","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_biomarker_wip")\

ner_oncology_biomarker_wip_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_oncology_biomarker_wip"])\
    .setOutputCol("ner_oncology_biomarker_wip_chunk")

ner_stages = [document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    ner_oncology_wip,
    ner_oncology_wip_converter,
    ner_oncology_tnm_wip,
    ner_oncology_tnm_wip_converter,
    ner_oncology_biomarker_wip,
    ner_oncology_biomarker_wip_converter]

ner_pipeline = nlp.Pipeline(stages=ner_stages)

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_model = ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_oncology_wip download started this may take some time.
[OK!]
ner_oncology_tnm_wip download started this may take some time.
[OK!]
ner_oncology_biomarker_wip download started this may take some time.
[OK!]


In [0]:
ner_oncology_wip_labels = list(set([label.split('-')[-1] for label in ner_oncology_wip.getClasses() if label != 'O']))

len(ner_oncology_wip_labels)

Out[5]: 49

In [0]:
label_df = pd.DataFrame()
for column in range((len(ner_oncology_wip_labels)//10)+1):
  label_df = pd.concat([label_df, pd.DataFrame(ner_oncology_wip_labels, columns = [''])[column*10:(column+1)*10].reset_index(drop= True)], axis = 1)

label_df.fillna('')

0 
 Biomarker 
 Site_Liver 
 Hormonal_Therapy 
 Chemotherapy 
 Death_Entity 
 
 
 1 
 Grade 
 Date 
 Cycle_Day 
 Site_Lung 
 Site_Brain 
 
 
 2 
 Pathology_Result 
 Imaging_Test 
 Age 
 Site_Breast 
 Invasion 
 
 
 3 
 Tumor_Finding 
 Cycle_Number 
 Relative_Date 
 Gender 
 Histological_Type 
 
 
 4 
 Unspecific_Therapy 
 Immunotherapy 
 Cycle_Count 
 Site_Other_Body_Part 
 Metastasis 
 
 
 5 
 Duration 
 Radiation_Dose 
 Cancer_Dx 
 Response_To_Treatment 
 Tumor_Size 
 
 
 6 
 Pathology_Test 
 Site_Lymph_Node 
 Direction 
 Frequency 
 Radiotherapy 
 
 
 7 
 Smoking_Status 
 Targeted_Therapy 
 Cancer_Score 
 Cancer_Surgery 
 Biomarker_Result 
 
 
 8 
 Oncogene 
 Performance_Status 
 Dosage 
 Race_Ethnicity 
 Staging 
 
 
 9 
 Line_Of_Therapy 
 Adenopathy 
 Route 
 Site_Bone

In [0]:
ner_oncology_tnm_wip_labels = list(set([label.split('-')[-1] for label in ner_oncology_tnm_wip.getClasses() if label != 'O']))

print(ner_oncology_tnm_wip_labels)

['Tumor_Description', 'Cancer_Dx', 'Tumor', 'Metastasis', 'Lymph_Node', 'Staging', 'Lymph_Node_Modifier']


In [0]:
ner_oncology_biomarker_wip_labels = list(set([label.split('-')[-1] for label in ner_oncology_biomarker_wip.getClasses() if label != 'O']))

print(ner_oncology_biomarker_wip_labels)

['Biomarker_Result', 'Biomarker']


In [0]:
sample_text_1 = '''A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total anterior hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma. Neoadjuvant chemotherapy with the regimens of Taxotere (75 mg/m2), Epirubicin (75 mg/m2), and Cyclophosphamide (500 mg/m2) was given for 6 cycles with poor response, followed by a modified radical mastectomy (MRM) with dissection of axillary lymph nodes and skin grafting. Postoperatively, radiotherapy was done with 5000 cGy in 25 fractions. The histopathologic examination revealed a metaplastic carcinoma with squamous differentiation associated with adenomyoepithelioma. Immunohistochemistry study showed that the tumor cells are positive for epithelial markers-cytokeratin (AE1/AE3) stain, and myoepithelial markers, including cytokeratin 5/6 (CK 5/6), p63, and S100 stains. Expressions of hormone receptors, including ER, PR, and Her-2/Neu, were all negative. The dissected axillary lymph nodes showed metastastic carcinoma with negative hormone receptors in 3 nodes. The patient was staged as pT3N1aM0, with histologic tumor grade III.'''

sample_text_2 = '''She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass. A Pap smear performed one month later was positive for atypical glandular cells suspicious for adenocarcinoma. The pathologic specimen showed extension of the tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged lymph nodes. The final pathologic diagnosis of the tumor was stage IIIC papillary serous ovarian adenocarcinoma. Two months later, the patient was diagnosed with lung metastases.'''

sample_text_3 = '''In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9, CD10, CD13, CD19, CD20, CD34, CD38, CD58, CD66c, CD123, HLA-DR, cCD79a, and TdT on flow cytometry.

Measurements of serum tumor markers showed elevated level of cytokeratin 19 fragment (Cyfra21-1: 4.77 ng/mL), neuron-specific enolase (NSE: 19.60 ng/mL), and squamous cell carcinoma antigen (SCCA: 2.58 ng/mL). The results were negative for serum carbohydrate antigen 125 (CA125), carcinoembryonic antigen (CEA) and vascular endothelial growth factor (VEGF). Immunohistochemical staining showed positive staining for CK5/6, P40 and PD-L1 (+ 80% tumor cells), and negative staining for TTF-1, PD-1 and weakly positive staining for ALK. Molecular analysis indicated no EGFR mutation or ROS1 fusion.'''

In [0]:
data = spark.createDataFrame(pd.DataFrame([sample_text_1, sample_text_2, sample_text_3], columns = ['text']))

In [0]:
results = ner_model.transform(data).collect()

In [0]:
visualiser = nlp.viz.NerVisualizer()

In [0]:
ner_html_1 = visualiser.display(results[0], label_col='ner_oncology_wip_chunk', return_html=True)

displayHTML(ner_html_1)

A 65-year-old Age woman Gender had a history of debulking surgery Cancer_Surgery , bilateral Direction oophorectomy Cancer_Surgery with omentectomy Cancer_Surgery , total anterior hysterectomy Cancer_Surgery with radical pelvic lymph nodes dissection Cancer_Surgery due to ovarian carcinoma Cancer_Dx ( mucinous-type Histological_Type carcinoma Cancer_Dx , stage Ic Staging ) 1 year ago Relative_Date . Patient's medical compliance was poor and failed to complete her Gender chemotherapy Chemotherapy ( cyclophosphamide Chemotherapy 750 mg/m2 Dosage , carboplatin Chemotherapy 300 mg/m2 Dosage ). Recently, she Gender noted a palpable right Direction breast Site_Breast mass Tumor_Finding , 15 cm Tumor_Size in size which nearly occupied the whole right Direction breast Site_Breast in 2 months Relative_Date . Core needle biopsy Pathology_Test revealed metaplastic Histological_Type carcinoma Cancer_Dx . Neoadjuvant chemotherapy Chemotherapy with the regimens of Taxotere Chemotherapy ( 75 mg/m2 Dosage ), Epirubicin Chemotherapy ( 75 mg/m2 Dosage ), and Cyclophosphamide Chemotherapy ( 500 mg/m2 Dosage ) was given for 6 cycles Cycle_Count with poor response Response_To_Treatment , followed by a modified radical mastectomy Cancer_Surgery ( MRM Imaging_Test ) with dissection Cancer_Surgery of axillary lymph nodes Site_Lymph_Node and skin grafting Cancer_Surgery . Postoperatively, radiotherapy Radiotherapy was done with 5000 cGy Radiation_Dose in 25 fractions Cycle_Count . The histopathologic examination Pathology_Test revealed a metaplastic Histological_Type carcinoma Cancer_Dx with squamous differentiation Pathology_Result associated with adenomyoepithelioma Cancer_Dx . Immunohistochemistry study Pathology_Test showed that the tumor cells Pathology_Result are positive Biomarker_Result for epithelial markers-cytokeratin Biomarker ( AE1/AE3 Biomarker ) stain, and myoepithelial markers Biomarker , including cytokeratin 5/6 Biomarker ( CK 5/6 Biomarker ), p63 Biomarker , and S100 Biomarker stains. Expressions of hormone receptors Biomarker , including ER Biomarker , PR Biomarker , and Her Gender - 2/Neu Biomarker , were all negative Biomarker_Result . The dissected axillary lymph nodes Site_Lymph_Node showed metastastic Metastasis carcinoma Cancer_Dx with negative Biomarker_Result hormone receptors Biomarker in 3 nodes Site_Lymph_Node . The patient was staged as pT3N1aM0 Staging , with histologic tumor Histological_Type grade III Grade .

In [0]:
ner_html_2 = visualiser.display(results[1], label_col='ner_oncology_tnm_wip_chunk', return_html=True)

displayHTML(ner_html_2)

She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass Tumor . A Pap smear performed one month later was positive for atypical glandular cells Tumor_Description suspicious for adenocarcinoma Cancer_Dx . The pathologic specimen showed extension Tumor_Description of the tumor Tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged Lymph_Node_Modifier lymph nodes Lymph_Node . The final pathologic diagnosis of the tumor Tumor was stage IIIC Staging papillary Tumor_Description serous Tumor_Description ovarian adenocarcinoma Cancer_Dx . Two months later, the patient was diagnosed with lung metastases Metastasis .

In [0]:
ner_html_3 = visualiser.display(results[2], label_col='ner_oncology_biomarker_wip_chunk', return_html=True)

displayHTML(ner_html_3)

In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive Biomarker_Result for CD9 Biomarker , CD10 Biomarker , CD13 Biomarker , CD19 Biomarker , CD20 Biomarker , CD34 Biomarker , CD38 Biomarker , CD58 Biomarker , CD66c Biomarker , CD123 Biomarker , HLA-DR Biomarker , cCD79a Biomarker , and TdT Biomarker on flow cytometry. Measurements of serum tumor markers Biomarker showed elevated Biomarker_Result level of cytokeratin 19 fragment Biomarker ( Cyfra21-1 Biomarker : 4.77 ng/mL Biomarker_Result ), neuron-specific enolase Biomarker ( NSE Biomarker : 19.60 ng/mL Biomarker_Result ), and squamous cell carcinoma antigen Biomarker ( SCCA Biomarker : 2.58 ng/mL Biomarker_Result ). The results were negative Biomarker_Result for serum carbohydrate antigen 125 Biomarker ( CA125 Biomarker ), carcinoembryonic antigen Biomarker ( CEA Biomarker ) and vascular endothelial growth factor Biomarker ( VEGF Biomarker ). Immunohistochemical staining showed positive Biomarker_Result staining for CK5/6 Biomarker , P40 Biomarker and PD-L1 Biomarker ( + Biomarker_Result 80% tumor cells), and negative Biomarker_Result staining for TTF-1 Biomarker , PD-1 Biomarker and weakly positive Biomarker_Result staining for ALK Biomarker . Molecular analysis indicated no EGFR mutation Biomarker or ROS1 Biomarker fusion.

## Relation Extraction Models

RE Models are used to link entities that are related. For oncology entities, you can use general models (such as re_oncology_granular_wip) or you can select a specific model depending on your needs (e.g. re_oncology_size_wip to link tumors and their sizes, or re_oncology_biomarker_result_wip to link biomarkers and their results).

In [0]:
pos_tagger = nlp.PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel.pretrained("dependency_conllu", "en") \
    .setInputCols(["sentence", "pos_tags", "token"]) \
    .setOutputCol("dependencies")

re_oncology_granular_wip = medical.RelationExtractionModel.pretrained("re_oncology_granular_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_wip_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_granular_wip") \
    .setRelationPairs(['Date-Cancer_Dx', 'Cancer_Dx-Date', 'Tumor_Finding-Site_Breast', 'Site_Breast-Tumor_Finding',
                       'Relative_Date-Tumor_Finding', 'Tumor_Fiding-Relative_Date', 'Tumor_Finding-Tumor_Size', 'Tumor_Size-Tumor_Finding',
                       'Pathology_Test-Cancer_Dx', 'Cancer_Dx-Pathology_Test']) \
    .setMaxSyntacticDistance(10)    

re_oncology_size_wip = medical.RelationExtractionModel.pretrained("re_oncology_size_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_wip_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_size_wip") \
    .setRelationPairs(['Tumor_Finding-Tumor_Size', 'Tumor_Size-Tumor_Finding']) \
    .setMaxSyntacticDistance(10)    

re_oncology_biomarker_result_wip = medical.RelationExtractionModel.pretrained("re_oncology_biomarker_result_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_biomarker_wip_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_biomarker_result_wip") \
    .setRelationPairs(['Biomarker-Biomarker_Result', 'Biomarker_Result-Biomarker']) \
    .setMaxSyntacticDistance(10)      

re_stages = ner_stages + [pos_tagger, dependency_parser, re_oncology_granular_wip, re_oncology_size_wip, re_oncology_biomarker_result_wip]

re_pipeline = nlp.Pipeline(stages=re_stages)

re_model = re_pipeline.fit(empty_data)

pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
re_oncology_granular_wip download started this may take some time.
Approximate size to download 261 KB
[OK!]
re_oncology_size_wip download started this may take some time.
Approximate size to download 261.3 KB
[OK!]
re_oncology_biomarker_result_wip download started this may take some time.
Approximate size to download 259.6 KB
[OK!]


In [0]:
sample_text_4 = '''Two years ago, she noted a palpable right breast mass, 15 cm in size. Core needle biopsy revealed metaplastic carcinoma.'''

sample_text_5 = '''The patient presented a 2 cm mass in her left breast, and the tumor in her other breast was 3 cm long.'''

sample_text_6 = '''Immunohistochemical staining showed positive staining for CK5/6, P40 and PD-L1, and negative staining for TTF-1, PD-1 and weakly positive staining for ALK. Immunohistochemistry study showed that the tumor cells are positive for epithelial markers-cytokeratin and myoepithelial markers, including cytokeratin 5/6, p63, and S100 stains.'''

In [0]:
re_data = spark.createDataFrame(pd.DataFrame([sample_text_4, sample_text_5, sample_text_6], columns = ['text']))

In [0]:
re_results = re_model.transform(re_data).collect()

In [0]:
re_visualiser = nlp.viz.RelationExtractionVisualizer()

In [0]:
re_html_1 = re_visualiser.display(re_results[0], relation_col='re_oncology_granular_wip', return_html=True)

displayHTML(re_html_1)

Two years ago RELATIVE_DATE , she noted a palpable right breast SITE_BREAST mass TUMOR_FINDING , 15 cm TUMOR_SIZE in size. Core needle biopsy PATHOLOGY_TEST revealed metaplastic carcinoma CANCER_DX . <polyline fill="none" points="596.0,55.0 595.3061224489795,53.79607986468223 594.6122448979592,52.593128713376224 593.9183673469388,51.39211553009376 593.2244897959184,50.194009298846574 592.5306122448981,48.999779003646445 591.8367346938776,47.810393628505125 591.1428571428572,46.626822157434425 590.4489795918367,45.450033574446024 589.7551020408164,44.280996863551756 589.0612244897959,43.12068100876335 588.3673469387754,41.97005499409259 587.6734693877551,40.83008780355124 586.9795918367347,39.70174842115105 586.2857142857143,38.586005830903794 585.591836734694,37.48382901682123 584.8979591836735,36.396186962915124 584.2040816326531,35.32404865319723 583.5102040816327,34.26838307167932 582.8163265306123,33.23015920237316 582.1224489795918,32.21034602929051 581.4285714285713,31.209912536443145 580.7346938775511,30.229827707842823 580.0408163265306,29.271060527501298 579.3469387755102,28.334579979430348 578.6530612244899,27.421355047641715 577.9591836734694,26.532354716147186 577.265306122449,25.668547968958514 576.5714285714287,24.83090379008747 575.8775510204082,24.020391163545803 575.1836734693878,23.237979073345294 574.4897959183673,22.484636503497697 573.795918367347,21.761332438014776 573.1020408163265,21.069035860908297 572.4081632653061,20.408715756190023 571.7142857142858,19.781341107871725 571.0204081632653,19.187880899965155 570.3265306122449,18.629304116482082 569.6326530612245,18.106579741434267 568.9387755102041,17.620676758833483 568.2448979591837,17.172564152691482 567.5510204081633,16.76321090702004 566.8571428571429,16.393586005830905 566.1632653061224,16.064658433135854 565.469387755102,15.77739717294665 564.7755102040817,15.53277120927505 564.0816326530612,15.33174952613282 563.3877551020408,15.175301107531727 562.6938775510204,15.064394937483533 562.0,15.0 562.0,15.0 561.3061224489795,14.982762284422305 560.6122448979592,15.012035801409274 559.9183673469388,15.086851566949145 559.2244897959184,15.20624059703015 558.530612244898,15.369233907640522 557.8367346938775,15.574862514768505 557.1428571428572,15.822157434402333 556.4489795918367,16.11014968253024 555.7551020408163,16.43787027514046 555.0612244897958,16.80435022822123 554.3673469387754,17.208620557760792 553.6734693877551,17.64971227974738 552.9795918367347,18.126656410169232 552.2857142857142,18.638483965014576 551.5918367346939,19.184225960271654 550.8979591836735,19.762913411928704 550.204081632653,20.37357733597396 549.5102040816327,21.01524874839565 548.8163265306123,21.686958665182026 548.1224489795917,22.387738102321304 547.4285714285713,23.116618075801746 546.734693877551,23.87262960161157 546.0408163265305,24.654803695739016 545.3469387755101,25.462171374172325 544.6530612244898,26.293763652899724 543.9591836734694,27.148611547909454 543.2653061224489,28.02574607518976 542.5714285714286,28.92419825072886 541.8775510204082,29.842999090515008 541.1836734693877,30.78117961053642 540.4897959183673,31.737770826781354 539.7959183673469,32.71180375523804 539.1020408163265,33.7023094118947 538.408163265306,34.70831881273959 537.7142857142858,35.728862973760926 537.0204081632653,36.762972910946964 536.3265306122448,37.809679640285935 535.6326530612245,38.868014177766064 534.9387755102041,39.93700753937559 534.2448979591836,41.015690741102766 533.5510204081633,42.103094798935814 532.8571428571429,43.19825072886297 532.1632653061224,44.300189546872474 531.4693877551019,45.40794226895255 530.7755102040816,46.52053991109146 530.0816326530612,47.63701348927743 529.3877551020407,48.75639401949867 528.6938775510204,49.877712517743454 528.0,51.0" stroke="#506991" stroke-width="1" /> is_location_of <polyline fill="none" points="688.0,55.0 687.2653061224488,53.79607986468223 686.530612244898,52.593128713376224 685.795918367347,51.39211553009376 685.0612244897959,

In [0]:
re_html_2 = re_visualiser.display(re_results[1], relation_col='re_oncology_size_wip', return_html=True)

displayHTML(re_html_2)

The patient presented a 2 cm TUMOR_SIZE mass TUMOR_FINDING in her left breast, and the tumor TUMOR_FINDING in her other breast was 3 cm TUMOR_SIZE long. <polyline fill="none" points="352.0,55.0 351.4081632653061,53.79607986468223 350.8163265306122,52.593128713376224 350.2244897959185,51.39211553009376 349.6326530612246,50.194009298846574 349.0408163265306,48.999779003646445 348.4489795918367,47.810393628505125 347.85714285714295,46.626822157434425 347.26530612244903,45.450033574446024 346.6734693877551,44.280996863551756 346.0816326530612,43.12068100876335 345.48979591836735,41.97005499409259 344.8979591836735,40.83008780355124 344.3061224489796,39.70174842115105 343.7142857142857,38.586005830903794 343.12244897959187,37.48382901682123 342.530612244898,36.396186962915124 341.93877551020415,35.32404865319723 341.34693877551024,34.26838307167932 340.7551020408163,33.23015920237316 340.1632653061224,32.21034602929051 339.57142857142856,31.209912536443145 338.9795918367347,30.229827707842823 338.3877551020408,29.271060527501298 337.7959183673469,28.334579979430348 337.204081632653,27.421355047641715 336.61224489795916,26.532354716147186 336.02040816326536,25.668547968958514 335.42857142857144,24.83090379008747 334.8367346938776,24.020391163545803 334.24489795918373,23.237979073345294 333.6530612244898,22.484636503497697 333.0612244897959,21.761332438014776 332.46938775510205,21.069035860908297 331.8775510204082,20.408715756190023 331.28571428571433,19.781341107871725 330.6938775510204,19.187880899965155 330.1020408163265,18.629304116482082 329.51020408163265,18.106579741434267 328.9183673469388,17.620676758833483 328.32653061224494,17.172564152691482 327.734693877551,16.76321090702004 327.14285714285717,16.393586005830905 326.55102040816325,16.064658433135854 325.9591836734694,15.77739717294665 325.36734693877554,15.53277120927505 324.7755102040816,15.33174952613282 324.18367346938777,15.175301107531727 323.5918367346939,15.064394937483533 323.0,15.0 323.0,15.0 322.40816326530614,14.982762284422305 321.81632653061223,15.012035801409274 321.2244897959184,15.086851566949145 320.63265306122446,15.20624059703015 320.0408163265306,15.369233907640522 319.4489795918367,15.574862514768505 318.85714285714295,15.822157434402333 318.26530612244903,16.11014968253024 317.6734693877551,16.43787027514046 317.08163265306115,16.80435022822123 316.48979591836735,17.208620557760792 315.8979591836734,17.64971227974738 315.3061224489796,18.126656410169232 314.71428571428567,18.638483965014576 314.12244897959187,19.184225960271654 313.53061224489795,19.762913411928704 312.9387755102041,20.37357733597396 312.3469387755102,21.01524874839565 311.75510204081627,21.686958665182026 311.16326530612236,22.387738102321304 310.57142857142856,23.116618075801746 309.9795918367347,23.87262960161157 309.3877551020408,24.654803695739016 308.7959183673469,25.462171374172325 308.2040816326531,26.293763652899724 307.61224489795916,27.148611547909454 307.02040816326524,28.02574607518976 306.42857142857144,28.92419825072886 305.83673469387753,29.842999090515008 305.2448979591837,30.78117961053642 304.65306122448976,31.737770826781354 304.0612244897959,32.71180375523804 303.46938775510205,33.7023094118947 302.87755102040813,34.70831881273959 302.28571428571433,35.728862973760926 301.69387755102036,36.762972910946964 301.1020408163265,37.809679640285935 300.51020408163265,38.868014177766064 299.9183673469388,39.93700753937559 299.32653061224494,41.015690741102766 298.73469387755097,42.103094798935814 298.1428571428571,43.19825072886297 297.55102040816325,44.300189546872474 296.9591836734694,45.40794226895255 296.36734693877554,46.52053991109146 295.7755102040816,47.63701348927743 295.18367346938777,48.75639401949867 294.59183673469386,49.877712517743454 294.0,51.0" stroke="#208CB7" stroke-width="1" /> is_size_of <polyline fill="none" points="85.0,155.0 91.0204081632653,154.33673469387753 97.04081632653059,153.6734693877551 103.06122448979593,153.01020408163268 109.081632653

In [0]:
re_html_3 = re_visualiser.display(re_results[2], relation_col='re_oncology_biomarker_result_wip', return_html=True)

displayHTML(re_html_3)

Immunohistochemical staining showed positive BIOMARKER_RESULT staining for CK5/6 BIOMARKER , P40 BIOMARKER and PD-L1 BIOMARKER , and negative BIOMARKER_RESULT staining for TTF-1 BIOMARKER , PD-1 BIOMARKER and weakly positive BIOMARKER_RESULT staining for ALK BIOMARKER . Immunohistochemistry study showed that the tumor cells are positive BIOMARKER_RESULT for epithelial markers-cytokeratin BIOMARKER and myoepithelial markers BIOMARKER , including cytokeratin 5/6 BIOMARKER , p63 BIOMARKER , and S100 BIOMARKER stains. <polyline fill="none" points="629.0,55.0 626.9183673469387,53.79607986468223 624.8367346938775,52.593128713376224 622.7551020408164,51.39211553009376 620.6734693877551,50.194009298846574 618.5918367346939,48.999779003646445 616.5102040816327,47.810393628505125 614.4285714285714,46.626822157434425 612.3469387755104,45.450033574446024 610.265306122449,44.280996863551756 608.1836734693877,43.12068100876335 606.1020408163265,41.97005499409259 604.0204081632652,40.83008780355124 601.938775510204,39.70174842115105 599.8571428571429,38.586005830903794 597.7755102040818,37.48382901682123 595.6938775510204,36.396186962915124 593.6122448979592,35.32404865319723 591.530612244898,34.26838307167932 589.4489795918367,33.23015920237316 587.3673469387754,32.21034602929051 585.2857142857142,31.209912536443145 583.2040816326531,30.229827707842823 581.1224489795918,29.271060527501298 579.0408163265307,28.334579979430348 576.9591836734694,27.421355047641715 574.8775510204081,26.532354716147186 572.795918367347,25.668547968958514 570.7142857142858,24.83090379008747 568.6326530612245,24.020391163545803 566.5510204081633,23.237979073345294 564.4693877551022,22.484636503497697 562.3877551020408,21.761332438014776 560.3061224489795,21.069035860908297 558.2244897959183,20.408715756190023 556.1428571428572,19.781341107871725 554.0612244897959,19.187880899965155 551.9795918367347,18.629304116482082 549.8979591836735,18.106579741434267 547.8163265306123,17.620676758833483 545.7346938775511,17.172564152691482 543.6530612244899,16.76321090702004 541.5714285714287,16.393586005830905 539.4897959183673,16.064658433135854 537.4081632653061,15.77739717294665 535.3265306122449,15.53277120927505 533.2448979591837,15.33174952613282 531.1632653061224,15.175301107531727 529.0816326530612,15.064394937483533 527.0,15.0 527.0,15.0 524.9183673469387,14.982762284422305 522.8367346938775,15.012035801409274 520.7551020408164,15.086851566949145 518.6734693877551,15.20624059703015 516.5918367346939,15.369233907640522 514.5102040816327,15.574862514768505 512.4285714285716,15.822157434402333 510.34693877551024,16.11014968253024 508.265306122449,16.43787027514046 506.18367346938766,16.80435022822123 504.1020408163265,17.208620557760792 502.0204081632652,17.64971227974738 499.9387755102041,18.126656410169232 497.85714285714283,18.638483965014576 495.7755102040817,19.184225960271654 493.6938775510205,19.762913411928704 491.6122448979592,20.37357733597396 489.53061224489795,21.01524874839565 487.44897959183675,21.686958665182026 485.3673469387754,22.387738102321304 483.2857142857142,23.116618075801746 481.20408163265313,23.87262960161157 479.12244897959187,24.654803695739016 477.0408163265306,25.462171374172325 474.9591836734694,26.293763652899724 472.8775510204081,27.148611547909454 470.795918367347,28.02574607518976 468.7142857142858,28.92419825072886 466.63265306122446,29.842999090515008 464.55102040816325,30.78117961053642 462.4693877551021,31.737770826781354 460.38775510204084,32.71180375523804 458.3061224489796,33.7023094118947 456.2244897959183,34.70831881273959 454.14285714285717,35.728862973760926 452.0612244897959,36.762972910946964 449.9795918367347,37.809679640285935 447.8979591836735,38.868014177766064 445.8163265306123,39.93700753937559 443.734693877551,41.015690741102766 441.65306122448976,42.103094798935814 439.57142857142856,43.19825072886297 437.48979591836735,44.300189546872474 435.40816326530614,45.40794226895255 433.32653061224494,46.52053991109146 43

## Assertion Status Models

With assertion status models, you will be able to identify if entities included in texts are mentioned as something present, absent, hypothetical, possible, etc. You can either try using the general assertion_oncology_wip model, or other models that are recommended for specific entity groups (such as assertion_oncology_problem_wip, which should be used for problem entities like Cancer_Dx or Metastasis).

In [0]:
assertion_oncology_wip = medical.AssertionDLModel.pretrained("assertion_oncology_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_wip_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_wip")

assertion_oncology_problem_wip = medical.AssertionDLModel.pretrained("assertion_oncology_problem_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_tnm_wip_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_problem_wip")

assertion_oncology_treatment_binary_wip = medical.AssertionDLModel.pretrained("assertion_oncology_treatment_binary_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_wip_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_treatment_binary_wip")

assertion_stages = ner_stages + [assertion_oncology_wip, assertion_oncology_problem_wip, assertion_oncology_treatment_binary_wip]

assertion_pipeline = nlp.Pipeline(stages=assertion_stages)

assertion_model = assertion_pipeline.fit(empty_data)

assertion_oncology_wip download started this may take some time.
[OK!]
assertion_oncology_problem_wip download started this may take some time.
[OK!]
assertion_oncology_treatment_binary_wip download started this may take some time.
[OK!]


In [0]:
sample_text_7 = 'The patient is suspected to have colorectal cancer. Family history is positive for other cancers. The result of the biopsy was positive. A CT scan was ordered to rule out metastases.'

sample_text_8 = 'The patient was diagnosed with breast cancer. She was suspected to have metastases in her lungs. Her family history is positive for ovarian cancer.'

sample_text_9 = 'The patient underwent a mastectomy. We recommend to start radiotherapy. The patient refused to chemotherapy.'

In [0]:
assertion_data = spark.createDataFrame(pd.DataFrame([sample_text_7, sample_text_8, sample_text_9], columns = ['text']))

In [0]:
assertion_results = assertion_model.transform(assertion_data).collect()

In [0]:
assertion_visualiser = nlp.viz.AssertionVisualizer()

In [0]:
assert_html_1 = assertion_visualiser.display(assertion_results[0], label_col ='ner_oncology_wip_chunk', assertion_col='assertion_oncology_wip', return_html=True)

displayHTML(assert_html_1)

The patient is suspected to have colorectal cancer Cancer_Dx Possible . Family history is positive for other cancers Cancer_Dx Family . The result of the biopsy Pathology_Test Past was positive. A CT scan Imaging_Test Past was ordered to rule out metastases Metastasis Possible .

In [0]:
assert_html_2 = assertion_visualiser.display(assertion_results[1], label_col ='ner_oncology_tnm_wip_chunk', assertion_col='assertion_oncology_problem_wip', return_html=True)

displayHTML(assert_html_2)

The patient was diagnosed with breast cancer Cancer_Dx Medical_History . She was suspected to have metastases Metastasis Possible in her lungs. Her family history is positive for ovarian cancer Cancer_Dx Family_History .

In [0]:
assert_html_3 = assertion_visualiser.display(assertion_results[2], label_col ='ner_oncology_wip_chunk', assertion_col='assertion_oncology_treatment_binary_wip', return_html=True)

displayHTML(assert_html_3)

The patient underwent a mastectomy Cancer_Surgery Present_Or_Past . We recommend to start radiotherapy Radiotherapy Hypothetical_Or_Absent . The patient refused to chemotherapy Chemotherapy Hypothetical_Or_Absent .